In [1]:
import fitz  # PyMuPDF
import docx  # python-docx for DOCX
from PIL import Image  # for image handling
import pytesseract  # Tesseract OCR for text extraction from images
from io import BytesIO
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import psycopg2
from flask import Flask, request, jsonify
import base64
from flask_cors import CORS
import spacy

c:\Users\DELL\anaconda3\envs\houda\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
# Charger le modèle de embeddings multilingue
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

In [3]:
app = Flask(__name__)
CORS(app, resources={r"/similarity/*": {"origins": "http://localhost:4200"}})

nlp_fr = spacy.load('fr_core_news_sm')
nlp_en = spacy.load('en_core_web_sm')

# Liste des diplômes
DEGREES = ["licence", "master","ingénieur"]

# Fonction pour extraire les diplômes d'un texte de CV
def extract_degree(resume_text):
    nlp_text = nlp_fr(resume_text)
    degrees = []
    for token in nlp_text:
        if token.text.lower() in DEGREES:
            degrees.append(token.text)
    for chunk in nlp_text.noun_chunks:
        chunk_text = chunk.text.lower().strip()
        if chunk_text in DEGREES:
            degrees.append(chunk.text)
    return [word.capitalize() for word in set(degrees)]

# Charger la base de données des compétences à partir d'un fichier externe
with open('linkedin') as f:
    external_source = list(f)

result = [element.strip().lower() for element in external_source]

# Fonction pour extraire les compétences d'un texte de CV
def extract_skill(resume_text):
    nlp_text = nlp_en(resume_text)
    tokens = [token.text for token in nlp_text if not token.is_stop]
    skills = result
    skillset = []
    for i in tokens:
        if i.lower() in skills:
            skillset.append(i)
    for i in nlp_text.noun_chunks:
        i = i.text.lower().strip()
        if i in skills:
            skillset.append(i)
    return [word.capitalize() for word in set([word.lower() for word in skillset])]

# Connexion à la base de données PostgreSQL
def connect_db():
    return psycopg2.connect(
        host='localhost',
        user='postgres',
        password='root',
        database='projet'
    )

def extract_text(file_path):
    if file_path.endswith('.pdf'):
        return extract_text_from_pdf(file_path)
    elif file_path.endswith('.docx'):
        return extract_text_from_docx(file_path)
    elif file_path.endswith('.jpg') or file_path.endswith('.png') or file_path.endswith('.tiff') or file_path.endswith('.gif'):
        return extract_text_from_image(file_path)
    else:
        return 'unknown'

# Fonction pour extraire le texte d'un document à partir d'un chemin de fichier    
def extract_text_from_docx(docx_path):
    doc = docx.Document("c:/file/"
+docx_path)
    text = "\n".join([para.text for para in doc.paragraphs])
    return text

# Fonction pour extraire le texte d'une image à partir d'un chemin de fichier
def extract_text_from_image(image_path):
    img = Image.open("c:/file/"
+image_path)
    text = pytesseract.image_to_string(img)
    return text

# Fonction pour extraire le texte d'un PDF à partir d'un chemin de fichier
def extract_text_from_pdf(pdf_path):
    pdf_data = fitz.open("c:/file/"
+pdf_path)
    text = ""
    for page_num in range(len(pdf_data)):
        page = pdf_data.load_page(page_num)
        text += page.get_text()
    return text

# Fonction pour récupérer les informations sur un job par son ID
def get_job_by_id(cursor, job_id):
    cursor.execute("SELECT requirements, niveau FROM job WHERE id = %s", (job_id,))
    job = cursor.fetchone()
    return job

# Fonction pour récupérer les CVs pour une offre d'emploi spécifique
def get_cvs_by_job_id(cursor, job_id):
    cursor.execute("SELECT id, cv FROM job_application WHERE job_id = %s", (job_id,))
    cvs = cursor.fetchall()
    return cvs

# Fonction pour récupérer les détails des postulations par leurs IDs dans le bon ordre
def get_postulations_by_ids(cursor, postulation_ids):
    if not postulation_ids:
        return []

    format_strings = ','.join(['%s'] * len(postulation_ids))
    cursor.execute(f"""
        SELECT 
            ja.id, ja.cv, ja.datecandidateur, ja.status, ja.title,
            c.first_name, c.last_name, c.niveau, c.email, c.telephone,ja.condidate_id,ja.job_id
        FROM job_application ja
        JOIN candidat c ON ja.condidate_id = c.id
        WHERE ja.id IN ({format_strings})
        ORDER BY array_position(ARRAY[{format_strings}], ja.id)
    """, tuple(postulation_ids) + tuple(postulation_ids))
    postulations = cursor.fetchall()
    return postulations

@app.route('/similarity/<int:job_id>', methods=['GET'])
def calculate_similarity(job_id):
    db_conn = connect_db()
    cursor = db_conn.cursor()
    
    job = get_job_by_id(cursor, job_id)
    print("job",job)
    
    if not job:
        cursor.close()
        db_conn.close()
        return jsonify({'message': 'Job not found'}), 404
    
    # Générer les embeddings pour la description du job
    job_text = " ".join([str(item) for sublist in job for item in (sublist if isinstance(sublist, list) else [sublist])])
    job_embedding = model.encode([job_text])
    # Récupérer les CVs pour cette offre d'emploi
    cvs = get_cvs_by_job_id(cursor, job_id)
    print("cvs",cvs)
    
    if not cvs:
        cursor.close()
        db_conn.close()
        return jsonify({'message': 'No CVs found for this job'}), 404
    
    cv_texts = []
    for cv in cvs:
        cv_id, cv_blob = cv
        cv_text = extract_text(cv_blob)
     
        cv_degrees = extract_degree(cv_text)
        print("cv_degrees", cv_degrees)
    
        cv_skills = extract_skill(cv_text)
        print("cv_skills", cv_skills)
        cv_combined_text = " ".join(cv_degrees + cv_skills)
        cv_texts.append((cv_id, cv_combined_text))
    
    # Générer les embeddings pour les CVs
    cv_embeddings = model.encode([cv_text for _, cv_text in cv_texts])
    
    # Convertir les embeddings en matrices NumPy
    job_embedding = np.array(job_embedding)
    cv_embeddings = np.array(cv_embeddings)
    
    # Utiliser FAISS pour créer une instance de recherche de vecteurs pour les CVs
    index = faiss.IndexFlatL2(cv_embeddings.shape[1])
    index.add(cv_embeddings)
    
    # Calculer la similarité des CVs avec le job
    D, I = index.search(job_embedding, len(cv_embeddings))  # Trouver la similarité avec l'offre d'emploi
    
    # Afficher les IDs des CVs et leurs distances dans la console
    for idx, distance in zip(I[0], D[0]):
        print(f"CV ID: {cvs[idx][0]}, Distance: {distance}")
    
    # Trier les résultats par similarité
    results = sorted(zip(I[0], D[0]), key=lambda x: x[1])
    
    sorted_cv_ids = [cvs[idx][0] for idx, _ in results]
  
    # Récupérer les détails des postulations triées par similarité
    sorted_postulations = get_postulations_by_ids(cursor, sorted_cv_ids)
    
    # Convertir les résultats en un format sérialisable en JSON
    serialized_postulations = []
    for p in sorted_postulations:
        serialized_postulations.append({
           'id': p[0], 'cv': p[1], 'datecandidateur': p[2], 'status': p[3],
            'title': p[4], 'firstName': p[5], 'lastName': p[6], 'niveau': p[7],
            'email': p[8], 'telephone': p[9], 'candidat': p[10], 'job': p[11]
        })
    
    cursor.close()
    db_conn.close()
    return jsonify(serialized_postulations), 200

if __name__ == '__main__':
    app.run()



 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


job (['Git', 'Kubernetes', 'Docker', 'MySQL', 'Java'], 2)
cvs [(38, '20240909_174941_CV Feriel Ben Kraiem.pdf'), (39, '20240909_175146_CV_MOHAMED ALI MNASSAR.pdf'), (40, '20240909_175301_CV_Fkiri GHAZI.pdf'), (41, '20240909_175337_CV_Rim_Elhafi.pdf'), (42, '20240909_175419_CV_houda.pdf'), (37, '20240909_174830_CV_Wissem TRABELSSI.pdf')]
cv_degrees ['Ingénieur']
cv_skills ['Avancé', 'C++', 'Boot', 'Postgresql', 'Des', 'Jenkins', 'Zabbix', 'Python', 'Internet', 'Git', 'Hibernate', 'Jira', 'Mariadb', 'Ci', 'Apache', 'Firebase', 'Nginx', 'Application', 'Spring', 'Cd', 'Bitbucket', 'Github', 'Ssh', 'Intermédiaire', 'Dart', 'Ldap', 'Mobile', 'Mysql', 'Reconnaissance', 'Devops', 'Vagrant', 'Surveillance', 'Docker', 'Au', 'Web', 'Parole', 'Typescript', 'Stm', 'Java', 'C', 'Ansible', 'Reactjs', 'Angular']
cv_degrees ['Ingénieur']
cv_skills ['Openshift', 'Avancé', 'Virtualisation', 'Eureka', 'Computing', 'Des', 'F', 'O', 'Python', 'Maps', 'Box', 'Git', 'Passport', 'Hibernate', 'Json', 'Fog', 'Ku

127.0.0.1 - - [16/Sep/2024 20:32:03] "GET /similarity/10 HTTP/1.1" 200 -


CV ID: 37, Distance: 0.8053015470504761
CV ID: 38, Distance: 0.8195537328720093
CV ID: 40, Distance: 0.8750857710838318
CV ID: 42, Distance: 1.0594874620437622
CV ID: 39, Distance: 1.0600297451019287
CV ID: 41, Distance: 1.7523140907287598
